# IMPORTS

In [1]:
import sys
import os
dir_path = os.getcwd().split(os.path.sep)
root_index = dir_path.index('Machine_Learning_project')
root_path = os.path.sep.join(dir_path[:root_index + 1])
sys.path.append(root_path + '/code/')
sys.path.append(root_path + '/code/data_loaders/')
sys.path.append(root_path + '/code/Nostra_Neural_Network/')
sys.path.append(root_path + '/code/data_predictions/')



In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error
from Nostra_Neural_Network.metrics import *
from data_cup import *
import pandas as pd
from metrics import *
from sklearn.model_selection import train_test_split

### Ensemble Techniques Exploration
In this notebook, we explore ensemble techniques to amalgamate predictions from our most effective models, which were trained with the best-found hyperparameters during the model selection phase.

## Ensemble Strategies Explored
We examine two ensemble strategies:

1. **Arithmetic Average**
The arithmetic average entails aggregating predictions from each individual model by computing their mean.

2. **Weighted Average by performance**
The weighted average involves assigning distinct weights to each model's predictions before computing the aggregate. We chose to average the predictions of the different neural networks first, and then average this combined prediction with the SVM predictions. This approach aimed to leverage the complementary strengths of neural networks and SVMs, potentially enhancing the overall predictive performance of the ensemble.

## Experimental Approach
We conducted experiments by averaging several models making predictios in the validation set. It's worth noting that neural networks trained with the best-found hyperparameters during the model selection phase have already undergone an averaging process across five trials, specifically for SGD and Adam optimizers.


Additionally, the neural network trained using the gradient descent (GD) optimizer utilized a learning scheduler that required a validation set for training. Consequently, we split the training set into 10 folds, and for each fold, we trained the neural network on the remaining 9 folds. We then averaged the predictions made by each of these models on the test set

.




# DATA SPLITTING

In [4]:

df=cup = CupDataset('Cup_tr').data
y=df.iloc[:,-3:]
x=df.iloc[:,1:-3]


X_TR, X_TS, Y_TR, Y_TS = train_test_split(x, y, test_size=1/10, random_state=0)
X_TR, X_VL, Y_TR, Y_VL = train_test_split(X_TR, Y_TR, test_size=0.111, random_state=0)

Y_TR = Y_TR.values
Y_TS = Y_TS.values
Y_VL = Y_VL.values


# FINAL MODEL SELECTION
In this stage, we aggregate validation predictions from multiple models, computing both arithmetic and weighted averages. These models were trained on an initial training set comprising 800 examples and made predictions on a validation set containing 100 examples.

This step is crucial as it allows us to compare the performance of various models and ensemble approaches introduced earlier. Subsequently, we analyze the Mean Euclidean Error (MEE) alongside its standard deviation for each individual model and ensemble approach, aiding us in selecting the best-performing model.









In [ ]:

data_predictions_path = root_path + '/Models/data_predictions/'



Y_SVM_train = np.loadtxt(os.path.join(data_predictions_path, "y_pred_train_VL_svm.csv"), delimiter=",")
Y_SVM_test = np.loadtxt(os.path.join(data_predictions_path, "y_pred_test_VL_svm.csv"), delimiter=",")

Y_ADAM_train = np.loadtxt(os.path.join(data_predictions_path, "y_pred_train_VL_scikit.csv"), delimiter=",")
Y_ADAM_test = np.loadtxt(os.path.join(data_predictions_path, "y_pred_test_VL_scikit.csv"), delimiter=",")

Y_GD_train = np.loadtxt(os.path.join(data_predictions_path, "y_pred_train_VL_Antonio.csv"), delimiter=",")
Y_GD_test = np.loadtxt(os.path.join(data_predictions_path, "y_pred_test_VL_Antonio.csv"), delimiter=",")

Y_SGD_train = np.loadtxt(os.path.join(data_predictions_path, "y_pred_train_VL_SGD.csv"), delimiter=",")
Y_SGD_test = np.loadtxt(os.path.join(data_predictions_path, "y_pred_test_VL_SGD.csv"), delimiter=",")

# Compute the arithmetic average of all model predictions
Y_AVG_train = np.mean([Y_SVM_train, Y_ADAM_train, Y_GD_train, Y_SGD_train], axis=0)
Y_AVG_test = np.mean([Y_SVM_test, Y_ADAM_test, Y_GD_test, Y_SGD_test], axis=0)

# Compute the weighted average: average of neural networks first, then average with SVM
Y_NN_AVG_train = np.mean([Y_GD_train, Y_ADAM_train, Y_SGD_train], axis=0)
Y_NN_AVG_test = np.mean([Y_GD_test, Y_ADAM_test, Y_SGD_test], axis=0)
Y_W_AVG_train = np.mean([Y_NN_AVG_train, Y_SVM_train], axis=0)
Y_W_AVG_test = np.mean([Y_NN_AVG_test, Y_SVM_test], axis=0)

def format_metric(metric, std):
    return f"{metric:.2f} +- {std:.2f}"

def MSE(y, d):
    
    return mean_squared_error(y, d)

def MSE_std(y, d):
    mse = mean_squared_error(y, d)
    squared_diff = np.square(y - d)
    mse_std = np.sqrt(np.mean(np.square(squared_diff - mse)))
    return mse_std
print("--------------- Validation Train ---------------")
print("SGD MEE TRAIN:  ", format_metric(MEE(Y_SGD_train, Y_TR), MEE_std(Y_SGD_train, Y_TR)), "SGD MSE TRAIN:  ", format_metric(MSE(Y_SGD_train, Y_TR), MSE_std(Y_SGD_train, Y_TR)))
print("SVM MEE TRAIN:  ", format_metric(MEE(Y_SVM_train, Y_TR), MEE_std(Y_SVM_train, Y_TR)), "SVM MSE TRAIN:  ", format_metric(MSE(Y_SVM_train, Y_TR), MSE_std(Y_SVM_train, Y_TR)))
print("MLP ADAM MEE TRAIN:  ", format_metric(MEE(Y_ADAM_train, Y_TR), MEE_std(Y_ADAM_train, Y_TR)), "MLP ADAM MSE TRAIN:  ", format_metric(MSE(Y_ADAM_train, Y_TR), MSE_std(Y_ADAM_train, Y_TR)))
print("MLP GD scratch MEE TRAIN:  ", format_metric(MEE(Y_GD_train, Y_TR), MEE_std(Y_GD_train, Y_TR)), "MLP GD scratch MSE TRAIN:  ", format_metric(MSE(Y_GD_train, Y_TR), MSE_std(Y_GD_train, Y_TR)))
print("Arithmetic avg MEE TRAIN:  ", format_metric(MEE(Y_AVG_train, Y_TR), MEE_std(Y_AVG_train, Y_TR)), "Arithmetic avg MSE TRAIN:  ", format_metric(MSE(Y_AVG_train, Y_TR), MSE_std(Y_AVG_train, Y_TR)))
print("Weighted avg MEE TRAIN:  ", format_metric(MEE(Y_W_AVG_train, Y_TR), MEE_std(Y_W_AVG_train, Y_TR)), "Weighted avg MSE TRAIN:  ", format_metric(MSE(Y_W_AVG_train, Y_TR), MSE_std(Y_W_AVG_train, Y_TR)))

print("--------------- Validation Test ---------------")
print("SGD MEE TEST:  ", format_metric(MEE(Y_SGD_test, Y_VL), MEE_std(Y_SGD_test, Y_VL)), "SGD MSE test:  ", format_metric(MSE(Y_SGD_test, Y_VL), MSE_std(Y_SGD_test, Y_VL)))
print("SVM MEE TEST:  ", format_metric(MEE(Y_SVM_test, Y_VL), MEE_std(Y_SVM_test, Y_VL)), "SVM MSE test:  ", format_metric(MSE(Y_SVM_test, Y_VL), MSE_std(Y_SVM_test, Y_VL)))
print("MLP ADAM MEE TEST:  ", format_metric(MEE(Y_ADAM_test, Y_VL), MEE_std(Y_ADAM_test, Y_VL)), "MLP ADAM MSE test:  ", format_metric(MSE(Y_ADAM_test, Y_VL), MSE_std(Y_ADAM_test, Y_VL)))
print("MLP GD scratch MEE TEST:  ", format_metric(MEE(Y_GD_test, Y_VL), MEE_std(Y_GD_test, Y_VL)), "MLP GD scratch MSE test:  ", format_metric(MSE(Y_GD_test, Y_VL), MSE_std(Y_GD_test, Y_VL)))
print("Arithmetic avg MEE TEST:  ", format_metric(MEE(Y_AVG_test, Y_VL), MEE_std(Y_AVG_test, Y_VL)), "Arithmetic avg MSE test:  ", format_metric(MSE(Y_AVG_test, Y_VL), MSE_std(Y_AVG_test, Y_VL)))
print("Weighted avg MEE TEST:  ", format_metric(MEE(Y_W_AVG_test, Y_VL), MEE_std(Y_W_AVG_test, Y_VL)), "Weighted avg MSE test:  ", format_metric(MSE(Y_W_AVG_test, Y_VL), MSE_std(Y_W_AVG_test, Y_VL)))


# MODEL ASSESSTEMENT

In this section, we evaluate the performance of our best model, which is the weighted average ensemble approach. This model demonstrated the best performance during the validation phase, closely followed by the arithmetic average ensemble. For thoroughness, we also test the individual models on the internal test set to compare their performance and ensure the robustness of our ensemble approach.

In this phase, all models were retrained on the combined training set, which includes both the initial training set and the validation set used during the model selection phase.

In [ ]:
# Load test set predictions from different models
import os

data_predictions_path = root_path + '/Models/data_predictions/'

Y_SVM = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_SVR.csv"), delimiter=",")
Y_ADAM = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_scikit.csv"), delimiter=",")
Y_GD = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_Antonio.csv"), delimiter=",")
Y_SGD = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_SGD.csv"), delimiter=",")


# Compute the arithmetic average of all model predictions
Y_AVG = np.mean([Y_SVM, Y_ADAM, Y_GD, Y_SGD], axis=0)

# Compute the weighted average: average of selected models first, then average with SVM
Y_W_AVG = np.mean([np.mean([Y_GD, Y_ADAM, Y_SGD], axis=0), Y_SVM], axis=0)

# Define a helper function to format MEE and MEE_std with two decimal places
def format_metric(metric, std):
    return f"{metric:.3f} +- {std:.3f}"

print("--------------- MEAN SQUARED ERROR ---------------")
print("SGD : ", format_metric(mean_squared_error(Y_SGD, Y_TS), MSE_std(Y_SGD, Y_TS)))
print("SVM : ", format_metric(mean_squared_error(Y_SVM, Y_TS), MSE_std(Y_SVM, Y_TS)))
print("MLP ADAM : ", format_metric(mean_squared_error(Y_ADAM, Y_TS), MSE_std(Y_ADAM, Y_TS)))
print("MLP GD scratch : ", format_metric(mean_squared_error(Y_GD, Y_TS), MSE_std(Y_GD, Y_TS)))
print("Arithmetic : ", format_metric(mean_squared_error(Y_AVG, Y_TS), MSE_std(Y_AVG, Y_TS)))
print("Weighted avg : ", format_metric(mean_squared_error(Y_W_AVG, Y_TS), MSE_std(Y_W_AVG, Y_TS)))


# Evaluate and print MEE and its standard deviation for each model and the ensemble approaches
print("--------------- MEAN EUCLIDEAN ERROR ---------------")
print("SGD : ", format_metric(MEE(Y_SGD, Y_TS), MEE_std(Y_SGD, Y_TS)))
print("SVM : ", format_metric(MEE(Y_SVM, Y_TS), MEE_std(Y_SVM, Y_TS)))
print("MLP ADAM : ", format_metric(MEE(Y_ADAM, Y_TS), MEE_std(Y_ADAM, Y_TS)))
print("MLP GD scratch : ", format_metric(MEE(Y_GD, Y_TS), MEE_std(Y_GD, Y_TS)))
print("Arithmetic : ", format_metric(MEE(Y_AVG, Y_TS), MEE_std(Y_AVG, Y_TS)))
print("Weighted avg : ", format_metric(MEE(Y_W_AVG, Y_TS), MEE_std(Y_W_AVG, Y_TS)))


# FINAL RETRAINING AND BLIND TEST PREDICTIONS
Since we've already estimated the test error using the internal test set, we can proceed with a final re-training on all our development data. This ensures we utilize all available information for training without introducing bias from data leakage.

Based on our previous validation, the ensemble approach using the Weighted Average has shown the best performance. Therefore, we'll use this ensemble model as our final model for generating predictions on the blind test set

In [ ]:
data_predictions_path = root_path + '/Models/data_predictions/'

Y_SVM = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_SVR.csv"), delimiter=",")
Y_ADAM = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_scikit.csv"), delimiter=",")
Y_GD = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_Antonio.csv"), delimiter=",")
Y_SGD = np.loadtxt(os.path.join(data_predictions_path, "y_pred_internal_TS_SGD.csv"), delimiter=",")

# Compute the weighted average: average of selected models first, then average with SVM
Y_W_AVG = np.mean([np.mean([Y_GD, Y_ADAM, Y_SGD], axis=0), Y_SVM], axis=0)




# STORING BLIND RESULTS

In [53]:
import os

data_predictions_path = root_path + "/Models/datapredictions/"

# Store final blind test mean predictions
with open(os.path.join(data_predictions_path, 'BURDOS_ML-CUP23-TS.csv'), 'w') as outf:
    # Team Info
    outf.write("# Silvio Calderaro, Salvatore Ergoli, Antonio Brau\n")
    outf.write("# BURDOS\n")
    outf.write("# ML-CUP23 \n")
    outf.write("# 20/05/2024\n")

    # Writing predictions
    for i, pred in enumerate(Y_W_AVG, 1):
        outf.write(f"{i},{','.join(map(str, pred))}\n")
